In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from time import sleep
from selenium import webdriver

In [2]:
brand_name = 'Kia'

In [3]:
# 데이터프레임으로 읽어온다( columns = index, col1 )
df = pd.read_csv(f'{brand_name}_codes.csv')

In [4]:
df.head()

,codes
0,23004792
1,23004790
2,23004783
3,23004771
4,23004769


In [5]:
# def drop_dupCode 에서 중복값 제거 해줬기 때문에 바로 list
code_list = list(df['codes'])
print(f'{type(code_list)},{len(code_list)}')

<class 'list'>,33530


In [6]:
len(code_list)

33530

In [7]:
code_list.index(22523816)

27301

In [8]:
code_list = code_list[27302:]

In [9]:
len(code_list)

6228

In [10]:
base_url = 'https://www.kbchachacha.com/public/car/detail.kbc?carSeq='

In [11]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"}
driver = webdriver.Chrome("./chromedriver.exe")

for code in tqdm(code_list):
    sleep(4)
    url = f'{base_url}{code}' # code를 url에서 for문을 돌며 받아옴
    try: # url을 받아서 서버 상태를 받고
        res = requests.get(url, headers = headers)
        res.raise_for_status()
        driver.get(url)
        driver.implicitly_wait(10)
        server_check = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/strong')
        
    except Exception as e:
        print(f'{e}')
        sleep(4)
        pass # 에러가 나면 pass해서 다음 for문 돌려라
    
    else: # 서버에러가 없이 열린다면

        try: # 판매완료 xpath 받아올수 있니?
            sold_out = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div')
            sold_out.click()
            sleep(1.0)
            
        except: # 에러가 난다면 판매중이니까 파싱하자
            
            html = driver.page_source
            sleep(1.0)
            soup = BeautifulSoup(html,'html.parser')
            
            try:
                c_url = driver.current_url.split('=')[-1]
                full_name = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/strong').text
                price = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-price > div > dl > dd > strong').text
                use = soup.select_one('#divMarketPriceWrap > div > div.m-price-list02 > div.graph-area > strong > span').text[:-1]
                depreciation = soup.select_one('#divMarketPriceWrap > div > div.m-price-list02 > div.graph-area > strong > strong').text.split()[0]
                year = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-share > div.txt-info > span:nth-child(1)').text
                mileage = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-share > div.txt-info > span:nth-child(2)').text
                fuel = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-share > div.txt-info > span:nth-child(3)').text
                car_area = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-share > div.txt-info > span:nth-child(4)').text
                forecast = soup.select_one('#divMarketPriceWrap > div > div.m-price-list01 > div.cmm-price > strong').text
                sleep(1)
                c_type = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-detail-info > div > div.detail-info01 > table > tbody > tr:nth-child(4) > td:nth-child(2)').text
                car_cc = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-detail-info > div > div.detail-info01 > table > tbody > tr:nth-child(4) > td:nth-child(4)').text
                color = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-detail-info > div > div.detail-info01 > table > tbody > tr:nth-child(5) > td:nth-child(2)').text
                trans = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-detail-info > div > div.detail-info01 > table > tbody > tr:nth-child(3) > td:nth-child(2)').text
                loss = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500 > div.car-detail-info > div > div.detail-info02 > div > dl > dd:nth-child(2)').text.split()[0]
                flood = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500 > div.car-detail-info > div > div.detail-info02 > div > dl > dd:nth-child(4)').text.split()[0]
                usage = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500 > div.car-detail-info > div > div.detail-info02 > div > dl > dd:nth-child(6)').text.split()[0]
                change = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500 > div.car-detail-info > div > div.detail-info02 > div > dl > dd:nth-child(8)').text.split()[0]
                insurance = soup.select_one('#btnCarHistoryView2').text.split()[0]
                option = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-option > ul').text.split()[:-1]
                sales_corp = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.seller-info > div > div.dealer-location > div.map-txt > p:nth-child(1)').text.split(':')[1][1:]
                sales_loca = soup.select_one('#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.seller-info > div > div.dealer-location > div.map-txt > p:nth-child(2)').text.split(':')[1][1:]
                sleep(1)
                
            except:
                sleep(3)
                pass
            
            else:
                sleep(1)
                color = color.replace(',',' ')
                sales_loca = sales_loca.replace(',',' ')
                car_no = full_name.split(')')[0][1:]
                car_name = full_name.split('\n')[0].replace(f'({car_no})','')
                car_brand = car_name.split()[0]
                name_detailed = full_name.split('\n')[1]
                option = str(option).replace(',','/')
                option = option.replace("'","")
                option = option.replace("\n","/")
                price = price.replace(',','')
                mileage = mileage.replace(',','')
                forecast_min = forecast.split('~')[0].replace(',','')
                forecast_max = forecast.split('~')[1].replace(',','')
                car_cc = car_cc.replace(',','')
                
                f = open(f"{brand_name}_usedCar.csv", 'a')
                f.write(f'{str(c_url)},{car_area},{car_no},{car_brand},{car_name},{name_detailed},{price},{year},{use},{depreciation},{mileage},{fuel},{forecast_min},{forecast_max},{c_type},{car_cc},{color},{trans},{loss},{flood},{usage},{change},{insurance},{sales_corp},{sales_loca},{option}\n')
                
                f.close()

        else :
            sleep(3)
            pass
driver.close()
            # 받아올 수 있으면 아쉽게도 이미 팔렸으니 다음 단계로 pass 하자

C:\Users\ksy94\AppData\Local\Temp/ipykernel_13000/2356098939.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver.exe")
  0%|                                                                                         | 0/6228 [00:00<?, ?it/s]C:\Users\ksy94\AppData\Local\Temp/ipykernel_13000/2356098939.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  server_check = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/strong')
C:\Users\ksy94\AppData\Local\Temp/ipykernel_13000/2356098939.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  sold_out = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div')
C:\Users\ksy94\AppData\Local\Temp/ipykernel_13000/2356098939.py:34: DeprecationWar

# checkpoint 
- 27302/33530